# Simulated Effects Demo

This python notebook shows the simulated effects usage in the unified planning library and how to solve it using [scikit-decide](https://github.com/airbus/scikit-decide).

### Setup the library

First, we install unified_planning library and its dependencies from PyPi. Here, we use the `--pre` flag to use the latest development build.

In [7]:
pip install --pre unified-planning

Note: you may need to restart the kernel to use updated packages.


### Basic imports
The basic imports we need for this demo are abstracted in the `shortcuts` package.
We also need `up-skdecide`'s solver.

In [1]:
from unified_planning.shortcuts import *

import sys
sys.path.append("..")
from up_skdecide import *

### Problem definition

We start the problem modeling defining the `UserType` and the `Fluent`.

In [2]:
Location = UserType('Location')
Robot = UserType('Robot')

at = Fluent('at', Location, robot=Robot)
battery_charge = Fluent('battery_charge', IntType(0, 100), robot=Robot)

We define an action `move` with a simulated effect that models the battery consumption.

A `SimulatedEffect` instance can affect a list of fluent expressions, in this case only `battery_charge(robot)`.
The function `fun` performs the computation of the simulated effect decreasing the battery value by 10. This function receives as parameters the problem, the state in which the effect is applied, and the actual parameters of the action instance whose effect is being calculated.


In [3]:
move = InstantaneousAction('move', robot=Robot, l_from=Location, l_to=Location)
robot = move.parameter('robot')
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(Equals(at(robot), l_from))
move.add_precondition(GE(battery_charge(robot), 10))
move.add_precondition(Not(Equals(l_from, l_to)))
move.add_effect(at(robot), l_to)
def fun(problem, state, actual_params):
    value = state.get_value(battery_charge(actual_params.get(robot))).constant_value()
    return [Int(value - 10)]
move.set_simulated_effect(SimulatedEffect([battery_charge(robot)], fun))

Finally, we define the `Object` instances and, after creating the `Problem`, we set the initial values and the goal.

In [4]:
l1 = Object('l1', Location)
l2 = Object('l2', Location)
r1 = Object('r1', Robot)

problem = Problem('robot_with_simulated_effects')
problem.add_fluent(at)
problem.add_fluent(battery_charge)
problem.add_action(move)
problem.add_object(l1)
problem.add_object(l2)
problem.add_object(r1)

problem.set_initial_value(at(r1), l1)
problem.set_initial_value(battery_charge(r1), 100)

problem.add_goal(Equals(at(r1), l2))

### Solving the problem

We solve the problem automatically selecting a suitable planner.

In [5]:
from skdecide.hub.solver.iw import IW

In [6]:
with OneshotPlanner(problem_kind=problem.kind,
                    name='SkDecide',
                    params={
                        "solver": IW,
                        "config": {"state_features": lambda d, s: s},
                    },) as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))

[2022-04-22 18:47:54.886] [info] Running sequential IW solver from state [l1, 100]
[2022-04-22 18:47:54.886] [info] Running sequential IW(1) solver from state [l1, 100]
[2022-04-22 18:47:54.886] [info] Found a goal state: [l2, 100] (cost=1; best=1)
[2022-04-22 18:47:54.886] [info] IW(1) finished to solve from state [l1, 100] in 0.000618078 seconds.
[2022-04-22 18:47:54.886] [info] IW finished to solve from state [l1, 100] in 0.000653735 seconds.


KeyError: action move_r1_l1_l2 {
    preconditions = [
      (at(r1) == l1)
    ]
    effects = [
      at(r1) := l2
    ]
    simulated effect = [battery_charge(r1)] := simulated
  }